In [1]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as K

from keras_xlnet import PretrainedList, get_pretrained_paths

/home/chirag/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chirag/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chirag/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chirag/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
import os
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI
with tf.Session() as session:

    checkpoint_path = '/home/chirag/Downloads/cased_L-24_H-1024_A-16/xlnet_cased_L-24_H-1024_A-16'

    tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))
    model = load_trained_model_from_checkpoint(
        config_path=os.path.join(checkpoint_path, 'xlnet_config.json'),
        checkpoint_path=os.path.join(checkpoint_path, 'xlnet_model.ckpt'),
        batch_size=16,
        memory_len=512,
        target_len=128,
        in_train_phase=False,
        attention_type=ATTENTION_TYPE_BI,
    )
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

W0813 11:32:11.660554 140426944681792 deprecation_wrapper.py:119] From /home/chirag/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 11:32:11.664578 140426944681792 deprecation_wrapper.py:119] From /home/chirag/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 11:32:11.692194 140426944681792 deprecation_wrapper.py:119] From /home/chirag/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0813 11:32:11.804748 140426944681792 deprecation_wrapper.py:119] From /home/chirag/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0813 11:32:17.360063 14042694468

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 128, 1024),  32768000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 128)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 128, 1024)    0           Embed-Token[0][0]                
                                                                 Masking[0][0]                    
__________

In [15]:
import psycopg2
import pandas.io.sql as sqlio
import numpy as np

def get_dataframe_sql():
    df = None
    sql = "select emotion as label, text_ as text from dataset_emotion_only"
    con = None
    try:
        con = psycopg2.connect("host='35.200.234.61' dbname='sales' user='postgres' password='cx6ac54nmgGtLD1y'")
        df = sqlio.read_sql_query(sql, con)
    except psycopg2.DatabaseError as e:
        if con:
            con.rollback()
        print(e)
        sys.exit(1)
    finally:
        if con:
            con.close()
    df = df.sample(frac=1.0)
    df.label = df.label.astype('category')
    return df
  
df = get_dataframe_sql()
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_test = df[~msk]
df_train.head()

,label,text
4167,worry,who what? call me
26488,love,and you. okaaaay
17563,neutral,well... i don't drink
4185,worry,our neighbors cat died i really loved him
4012,happiness,well that was an interesting attempt at the co...


In [17]:
# # Load pre-trained model
# model = load_trained_model_from_checkpoint(
#     config_path=config_path,
#     checkpoint_path=model_path,
#     batch_size=1,
#     memory_len=0,
#     target_len=7,
#     in_train_phase=False,
#     attention_type=ATTENTION_TYPE_BI,
# )

In [18]:
category_counts = len(df_train.label.cat.categories)
category_counts

14

In [27]:
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [28]:
train_label[:3]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]], dtype=int8)

In [29]:
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [31]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=5,
            batch_size=32)
    model.save_weights('./emotion-detection-elmo-keras-xlnet-model.h5')

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([['who what? call me'],
       ['and you. okaaaay'],
       ["well... i don't drink"],
       ...,
       ["doesnt have a hangover and is getting ready for a good ol' english fry up"],
       [...

In [ ]:
new_text = [ "The bottle is blue in color",
            "I dont like you so much",
            "I had an amazing day at the stadium",
            "It was super fun after playing football",
            "my computer works fine",
            "I was shocked when I heard the airplane got crashed",
            "What the fuck!!!!!!",
            "What is your name?",
            "I was surprised when she got a gold medal for India.",
           "this website gave me a virus when i opened it more windows kept popping up",
           "the storm is here and the electricity is gone"]
new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./emotion-detection-elmo-keras-xlnet-model.h5')  
    predicts = model.predict(new_text, batch_size=32)

In [ ]:
# text = "Alls right with the world."
tokens = tokenizer.encode(train_text)

token_input = np.expand_dims(np.array(tokens), axis=0)
segment_input = np.zeros_like(token_input)
memory_length_input = np.zeros((1, 1))

In [ ]:
# Predict
results = model.predict_on_batch([token_input, segment_input, memory_length_input])
print('# Bi-directional')
for i in range(len(tokens)):
    print(results[0, i, :5])